In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from datetime import datetime, timedelta

%matplotlib inline
sns.set()

In [ ]:
conn = sqlite3.connect('../Data/db/fraude_detection_warehouse_.db')
transactions = pd.read_sql("SELECT * FROM transactions", conn)
transactions.head()

In [ ]:
customers = pd.read_sql("SELECT * FROM customers", conn)
customers.head()

In [ ]:
devices = pd.read_sql("SELECT * FROM devices", conn)
devices.head()

In [ ]:
locations = pd.read_csv("../Data/worldcities.csv")

## Feature engineering

determination des features à utiliser

In [ ]:
trans_features = ["transaction_id", "device_id", "customer_id", "amount", "transaction_type","status", "location"]
dev_features = ["device_id", "device_type", "os"]
cust_features = ["customer_id", "date_of_birth", "registration_date", "region", "email", "phone_number"]

extraire des différentes tables

In [ ]:
transactions = transactions[trans_features]
devices = devices[dev_features]
customers = customers[cust_features]

display(transactions.head())
print("\n")
display(devices.head())
print("\n")
display(customers.head())

calcul de l'age des customers

In [ ]:
date = datetime.strptime("2016-09-11", "%Y-%m-%d") - datetime.strptime("2016-05-15", "%Y-%m-%d")
date

In [ ]:
customers.date_of_birth = pd.to_datetime(customers.date_of_birth)

In [ ]:
customers.registration_date = pd.to_datetime(customers.registration_date)

In [ ]:
date_of_today = datetime.now()

def calcul_date(registration_date, date_of_birth):
    difference_registration = date_of_today - registration_date
    difference_birth = date_of_today - date_of_birth
    return int(difference_registration.days/360), int(difference_birth.days/360)

In [ ]:
customers[['year_since_registration', 'year_since_birth']] = customers.apply(
    lambda row: calcul_date(row['registration_date'], row['date_of_birth']), axis=1, result_type='expand'
)

In [ ]:
customers.drop(columns=["date_of_birth", "registration_date"], inplace=True)
customers.head()

nous utiliserons les régions comme localisation par la suite

In [ ]:
locations = locations[["city", "country", "admin_name"]]
locations = locations[locations.country.isin(["Cameroon", "Nigeria", "Gabon"])].drop(columns=["country"]).rename(columns={"city":"location"})
locations.head()

In [ ]:
transactions_ = pd.merge(transactions, locations, on="location")
transactions_.head() 

fusionner les dataframes

In [ ]:
data_1 = pd.merge(transactions_, customers, on="customer_id")
data_1.head()

In [ ]:
data_2 = pd.merge(data_1, devices, on="device_id").drop(columns=["device_id", "location"])
data_2.rename(columns={"admin_name":"transaction_location", "region" : "region_customer"}, inplace=True)
data_2.head()

In [ ]:
data = data_2.copy()

## Data preprocessing

In [ ]:
le = LabelEncoder()
columns = ["transaction_type", "status", "transaction_location", "region_customer", "device_type", "os"]
for column in columns:
    data[f"{column}_e"] = le.fit_transform(data[column])
data.head()

In [ ]:
scaler = StandardScaler()
data["amount_sc"] = scaler.fit_transform(data[["amount"]])
data.head()

In [ ]:
final_features =["transaction_type_e", "status_e", "transaction_location_e", "region_customer_e", "device_type_e", "os_e", "amount_sc"]
X  = data[final_features]

model = IsolationForest(contamination=0.2, random_state=42)
model.fit(X)

In [ ]:
data["anomalie"] = model.predict(X)
anomalie = data[data.anomalie==-1]
print(anomalie.shape)
anomalie.head()